In [4]:
library(Seurat)
library(scCustomize)
library(Azimuth)
library(archive)
library(ff)
library(readr)

Warning message:
“Your system is mis-configured: ‘/var/db/timezone/localtime’ is not a symlink”
Warning message:
“‘/var/db/timezone/localtime’ is not identical to any known timezone file”
Warning message:
“Failed to locate timezone database”
scCustomize v3.0.1
If you find the scCustomize useful please cite.
See 'samuel-marsh.github.io/scCustomize/articles/FAQ.html' for citation info.

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



Attaching shinyBS

Loading required package: bit


Attaching package: 'bit'


The following object is masked from 'package:base':

    xor


Attaching package ff

- getOption("fftempdir")=="/tmp/Rtmp7M1gUi/ff"


- getOption("ffextension")=="ff"


- getOption("ffdrop")==TRUE


- getOption("fffinonexit")==TRUE


- getOption("ffpagesize")==65536


- getOption("ffcaching")=="mmnoflush"  -- consider "ffeachflush" if your system stalls on large writes


- getOption("ffbatchbytes")==16777216 -- consider a d

In [2]:
# Leggiamo il dataset (questo è già preprocessato ma purtroppo in seurat v5 obj
list.files("dataset/rawdataset/GSE144744")

[1] "counts_gse144744.tar.gz"                       
[2] "GSE144744_metadata_per_cell.csv"               
[3] "RNA_counts"                                    
[4] "StillUnclearMetaAttachedAlmostFinishedLast.rds"

In [ ]:
archive_extract("dataset/rawdataset/GSE144744/counts_gse144744.tar.gz")

In [8]:
file.move("RNA_counts", "dataset/rawdataset/GSE144744/RNA_counts")

[1] TRUE

In [9]:
options(Seurat.object.assay.version = "v3") # FORZIAMO ALLA VERSIONE 3 DEGLI OGGETTI SEURAT

In [10]:
read_10x <- Read10X("dataset/rawdataset/GSE144744/RNA_counts")

In [12]:
seurat_obj <- CreateSeuratObject(read_10x)
#loading metadata
metadata <- read_csv("dataset/rawdataset/GSE144744/GSE144744_metadata_per_cell.csv")

Rows: 497705 Columns: 26
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (16): cell_names, donor, sample, sample_10X, HASH, match, batch_pair, g...
dbl   (9): nFeature_RNA, nCount_RNA, nCount_ADT, percent_mito, birth_year, a...
dttm  (1): sample_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [13]:
#adding metadata to obj
rm(read_10x)
seurat_obj <- AddMetaData(seurat_obj, metadata = metadata)
seurat_obj[["percent.mt"]] <- PercentageFeatureSet(seurat_obj, pattern = "^MT-")

Warning message:
"Setting row names on a tibble is deprecated."


In [14]:
# Puliamo
seurat_obj <- subset(seurat_obj, subset = nFeature_RNA < 2700 & nCount_RNA < 8000)

In [ ]:
seurat_split <- SplitObject(seurat_obj, split.by="sample")

# Facciamo girare Azimuth su ogni singolo campione singolarmente
seurat_split <- lapply(seurat_split, function(obj) {
    obj <- RunAzimuth(obj, ref='pbmcref')  # Sostituisci con il vero comando per Azimuth
    return(obj)
})


merged_data <- merge(x=seurat_split[[1]], y=seurat_split[-1])

Finding neighborhoods

Finding anchors

	Found 1596 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Predicting cell labels

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Predicting cell labels

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"

Integrating dataset 2 with reference dataset

Finding integration vectors

Integrating data

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from integrated_dr_ to integrateddr_"
Computing nearest neighbors

Running UMAP projection

Warning message in RunUMAP.default(object = neighborlist, reduction.model = reduction.model, :
"Number of neighbors between query and reference is not equal to the number of neighbors within reference"
23:31:32 Read 942 rows

23:31:32 Processing block 1 of 1

23:31:32 Commencing smooth kNN distance calibration using 1 thre

In [ ]:
# Sistemiamo un po' di metadati
merged_data$orig.ident <- merged_data$sample
merged_data$Condition <- ifelse(merged_data$prev_treatments == "none", "CTRL", "MS")
merged_data$Tissue.Type <- "PBMC"

In [ ]:
saveRDS(merged_data,'backup.rds')

In [1]:
gse144744 <- readRDS("backup.rds")

In [2]:
gse144744

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.4.1 but the current version is
4.4.2; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.5 but the current
version is 1.7.2; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t




An object of class Seurat 
15449 features across 491888 samples within 4 assays 
Active assay: RNA (15354 features, 0 variable features)
 2 layers present: counts, data
 3 other assays present: prediction.score.celltype.l1, prediction.score.celltype.l2, prediction.score.celltype.l3

In [11]:
# Convertiamo ad anndata
as.anndata(x = gse144744, file_path = ".", file_name = "gse144744_pbmc.h5ad")

ERROR: [1m[33mError[39m in `as.anndata()`:[22m
[1m[22m[33m![39m Necessary python package [32manndata[39m not found.
[36mℹ[39m Please install anndata and ensure reticulate is linked to correct python environment.
[36mℹ[39m After installation run `reticulate::py_module_available(module = 'anndata')` to confirm successful installation.


In [7]:
reticulate::py_module_available(module = 'anndata')

[1] FALSE

In [13]:
library(Seurat)
library(SeuratDisk)

# Supponendo che `gse144744` sia un oggetto Seurat
SaveH5Seurat(gse144744, filename = "gse144744_pbmc.h5Seurat")
Convert("gse144744_pbmc.h5Seurat", dest = "h5ad")


Attaching package: 'SeuratDisk'


The following object is masked from 'package:Azimuth':

    Connect


Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

No variable features found for RNA

No feature-level metadata found for RNA

Adding counts for prediction.score.celltype.l1

Adding data for prediction.score.celltype.l1

No variable features found for prediction.score.celltype.l1

No feature-level metadata found for prediction.score.celltype.l1

Adding counts for prediction.score.celltype.l2

Adding data for prediction.score.celltype.l2

No variable features found for prediction.score.celltype.l2

No feature-level metadata found for prediction.score.celltype.l2

Adding counts for prediction.score.celltype.l3

Adding data for prediction.score.celltype.l3

No variable features found for prediction.score.celltype.l3

No feature-level metadata found for prediction.score.celltype.l3

Validating h5Seurat file

Adding data from RNA as X

Adding coun